In [1]:
from datetime import datetime, time
import numpy as np
import os
import pandas as pd

In [2]:
# Set some global parameters
PATH = '/Users/cl/project/trading_project/'
FILE_NAME = 'j170520170103.csv'
DATA_STORE = 'store.h5'


## First, define some helper methods

In [3]:
def string_to_date(date_string):
    return datetime.strptime(date_string, '%Y%m%d%H%M%S')

def date_to_string(date):
    return date.strftime('%Y%m%d')

def calculate_stats(df):
    df['mid_price'] = (df['bid_price'] + df['ask_price']) / 2.
    df['vw_price'] = (df['bid_price'] * df['bid_volume'] + df['ask_price'] * df['ask_volume']) / (df['bid_volume'] + df['ask_volume'])
    return df

### Note that here I choose to use hd5f as the data store format instead of the .mat format

HD5F has native support with both pandas and matlab, greater scalability and much higher performance than .mat format. A detailed discussion can be seen [here](https://web.archive.org/web/20151120080623/http://matthewrocklin.com/blog/work/2015/03/16/Fast-Serialization/).

In addition, hd5f supports hierarchical data structure, which enables us to aggregate all the data ranging from all dates into one file instead of dispersing into different .mat file.

In [4]:
def save_data_as_hd5(contract_name, begin_time, end_time, df):
    filename = "{}_{}_{}".format(contract_name, begin_time, end_time)
    store = pd.HDFStore(DATA_STORE)
    store[filename] = df

## Now we are going to load the data

In [5]:
def LoadData(dir_name, file_name, begin_time=None, end_time=None):
    """
    Parameters
    ----------
    dir_name : directory path of the input file (string)
    file_name : file name of the input file (string)
    begin_time : in the format of yyyymmddhhmmss e.g., 20170103090000 (24-hour block)
    end_time: in the format of yyyymmddhhmmss e.g., 20170103090000 
    
    Returns
    -------
    data : a pandas DataFrame object
    """
    column_names = ['date','contract','time','mseconds','bid_price','bid_volume','ask_price','ask_volume','last_price',
                   'volume','avg_price','turn_over','mid_price','vw_price']
    path = os.path.join(dir_name, file_name)
    date_parser = lambda x: pd.datetime.strptime(x, '%Y%m%d %H:%M:%S %f')
    raw_data = pd.read_csv(path, names=column_names, index_col=False, parse_dates={'datetime': ['date','time','mseconds']}, date_parser=date_parser)
    data = calculate_stats(raw_data)
    
    begin_time = string_to_date(begin_time) if begin_time is not None else data.loc[data.index[0], 'datetime']
    end_time = string_to_date(end_time) if end_time is not None else data.loc[data.index[-1], 'datetime']
    
    # we do not consider any data before 9:00
    # TODO: Should we consider data after 15:00 ?
    data.set_index('datetime', inplace=True)
    date_filter = (data.index.hour >= 9) & (data.index >= begin_time) & (data.index <= end_time)
    data = data.loc[date_filter]
    save_data_as_hd5(data.loc[data.index[0], 'contract'], date_to_string(begin_time), date_to_string(end_time), data)
    return data
    



In [6]:
df = LoadData(PATH, FILE_NAME)
df.head()

,contract,bid_price,bid_volume,ask_price,ask_volume,last_price,volume,avg_price,turn_over,mid_price,vw_price
datetime,,,,,,,,,,,
2017-01-03 09:00:00.486,j1705,1519.0,126,1519.0,1,1520.0,4,151907.94,19140400.0,1519.00,1519.000000
2017-01-03 09:00:00.968,j1705,1518.5,188,1517.5,1,1518.5,10,151884.04,28554200.0,1518.00,1518.494709
2017-01-03 09:00:01.477,j1705,1518.0,236,1515.5,2,1516.5,1,151858.05,35838500.0,1516.75,1517.978992
2017-01-03 09:00:01.973,j1705,1517.0,306,1515.0,17,1517.0,4,151822.88,46457800.0,1516.00,1516.894737
2017-01-03 09:00:02.449,j1705,1517.5,414,1514.5,1,1516.5,1,151735.51,62818500.0,1516.00,1517.492771


## Now Let's make sure we can recover the data from the storage file

In [8]:
store = pd.HDFStore(DATA_STORE)
store.keys()

['/j1705_20170103_20170103']

In [9]:
df = store['j1705_20170103_20170103']
df.head()

,contract,bid_price,bid_volume,ask_price,ask_volume,last_price,volume,avg_price,turn_over,mid_price,vw_price
datetime,,,,,,,,,,,
2017-01-03 09:00:00.486,j1705,1519.0,126,1519.0,1,1520.0,4,151907.94,19140400.0,1519.00,1519.000000
2017-01-03 09:00:00.968,j1705,1518.5,188,1517.5,1,1518.5,10,151884.04,28554200.0,1518.00,1518.494709
2017-01-03 09:00:01.477,j1705,1518.0,236,1515.5,2,1516.5,1,151858.05,35838500.0,1516.75,1517.978992
2017-01-03 09:00:01.973,j1705,1517.0,306,1515.0,17,1517.0,4,151822.88,46457800.0,1516.00,1516.894737
2017-01-03 09:00:02.449,j1705,1517.5,414,1514.5,1,1516.5,1,151735.51,62818500.0,1516.00,1517.492771
